In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd
path = "/content/drive/My Drive/MTech AI/2nd SEM/NLP/Consumer_Complaints.csv"
data = pd.read_csv(path)
data = data.dropna()
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
298,3/1/2016,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt resulted from identity theft,I was called on my cell phone by First Nationa...,Company believes it acted appropriately as aut...,"First National Collection Bureau, Inc.",CA,921XX,Servicemember,Consent provided,Web,3/1/2016,Closed with explanation,Yes,No,1809977
314,8/13/2015,Debt collection,Medical,Communication tactics,Called after sent written cease of comm,I sent a cease and desist letter to a medical ...,Company believes complaint caused principally ...,"JP RECOVERY SERVICES, INC.",OH,440XX,Servicemember,Consent provided,Web,8/25/2015,Closed,Yes,Yes,1518155
1458,3/23/2017,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt was paid,"Phoenix Financial Services, LLC continues to r...",Company believes it acted appropriately as aut...,Phoenix Financial Services LLC,CO,802XX,Servicemember,Consent provided,Web,3/23/2017,Closed with explanation,Yes,No,2400971
1645,6/2/2016,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,ai broke some ribs while on a fishing trip to ...,Company believes it acted appropriately as aut...,"Credit Bureau Systems, Inc.",IL,629XX,Older American,Consent provided,Web,6/2/2016,Closed with explanation,Yes,No,1951609
2298,8/10/2015,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,Hi my name XXXX XXXX XXXX this company reporte...,Company believes it acted appropriately as aut...,ERC,CA,945XX,Servicemember,Consent provided,Web,8/10/2015,Closed with non-monetary relief,Yes,No,1511183


In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt



In [6]:
from nltk.corpus import stopwords
#nltk.download('stopwords')

import nltk
nltk.download('stopwords')
  

data = data['Consumer complaint narrative']
data = data.dropna()
stop_words = stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use','xxxx','xx'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def sent_to_words(sent):
    for s in sent:
        yield(gensim.utils.simple_preprocess(str(s),deacc=True))
data_words = list(sent_to_words(data))


In [0]:

print(data_words[:2])

[['was', 'called', 'on', 'my', 'cell', 'phone', 'by', 'first', 'national', 'debt', 'inc', 'and', 'was', 'told', 'owed', 'for', 'debt', 'incurred', 'xx', 'xx', 'xxxx', 'while', 'was', 'in', 'xxxx', 'the', 'debt', 'was', 'never', 'resolved', 'because', 'by', 'the', 'time', 'returned', 'home', 'from', 'xxxx', 'and', 'found', 'out', 'about', 'the', 'debt', 'xxxx', 'xxxx', 'was', 'no', 'longer', 'bank', 'or', 'it', 'moved', 'or', 'something', 'the', 'website', 'was', 'down', 'and', 'the', 'number', 'was', 'just', 'recording', 'then', 'find', 'out', 'that', 'xxxx', 'xxxx', 'was', 'bought', 'by', 'another', 'bank', 'and', 'this', 'debt', 'came', 'from', 'xx', 'xx', 'xxxx', 'or', 'xx', 'xx', 'xxxx', 'was', 'xxxx', 'xx', 'xx', 'xxxx', 'and', 'in', 'xxxx', 'xx', 'xx', 'xxxx', 'this', 'debt', 'is', 'nine', 'years', 'old'], ['sent', 'cease', 'and', 'desist', 'letter', 'to', 'medical', 'debt', 'collection', 'agency', 'for', 'bills', 'for', 'my', 'recently', 'deceased', 'husband', 'which', 'ca', 'af

In [0]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['was', 'called', 'on', 'my', 'cell', 'phone', 'by', 'first', 'national', 'debt', 'inc', 'and', 'was', 'told', 'owed', 'for', 'debt', 'incurred', 'xx', 'xx', 'xxxx', 'while', 'was', 'in', 'xxxx', 'the', 'debt', 'was', 'never', 'resolved', 'because', 'by', 'the', 'time', 'returned', 'home', 'from', 'xxxx', 'and', 'found', 'out', 'about', 'the', 'debt', 'xxxx', 'xxxx', 'was', 'no', 'longer', 'bank', 'or', 'it', 'moved', 'or', 'something', 'the', 'website', 'was', 'down', 'and', 'the', 'number', 'was', 'just', 'recording', 'then', 'find', 'out', 'that', 'xxxx', 'xxxx', 'was', 'bought', 'by', 'another', 'bank', 'and', 'this', 'debt', 'came', 'from', 'xx', 'xx', 'xxxx', 'or', 'xx', 'xx', 'xxxx', 'was', 'xxxx', 'xx', 'xx', 'xxxx', 'and', 'in', 'xxxx', 'xx', 'xx', 'xxxx', 'this', 'debt', 'is', 'nine', 'years', 'old']


In [0]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [0]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)

# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:2])

[['call', 'cell', 'phone', 'first', 'national', 'debt', 'inc', 'tell', 'owe', 'debt', 'incur', 'debt', 'never', 'resolve', 'time', 'return', 'home', 'find', 'debt', 'long', 'bank', 'move', 'something', 'website', 'number', 'recording', 'find', 'buy', 'bank', 'debt', 'come', 'debt', 'year', 'old'], ['send', 'cease', 'desist', 'letter', 'medical', 'debt', 'collection', 'agency', 'bill', 'recently', 'deceased', 'husband', 'ca', 'afford', 'pay', 'certified_mail', 'receipt', 'payment', 'make', 'never', 'get', 'receipt', 'back', 'get', 'robo', 'collection', 'call', 'day', 'day', 'week', 'want', 'call', 'stop']]


In [0]:
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 6), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]]


[[('bank', 2),
  ('buy', 1),
  ('call', 1),
  ('cell', 1),
  ('come', 1),
  ('debt', 6),
  ('find', 2),
  ('first', 1),
  ('home', 1),
  ('inc', 1),
  ('incur', 1),
  ('long', 1),
  ('move', 1),
  ('national', 1),
  ('never', 1),
  ('number', 1),
  ('old', 1),
  ('owe', 1),
  ('phone', 1),
  ('recording', 1),
  ('resolve', 1),
  ('return', 1),
  ('something', 1),
  ('tell', 1),
  ('time', 1),
  ('website', 1),
  ('year', 1)]]

In [0]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [0]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"assignment" + 0.025*"caller_id" + 0.024*"individual" + '
  '0.022*"family_member" + 0.022*"assign" + 0.018*"usaa" + 0.018*"parent" + '
  '0.016*"requirement" + 0.014*"ecmc" + 0.014*"buyer"'),
 (1,
  '0.142*"debt" + 0.098*"pay" + 0.085*"owe" + 0.045*"amount" + 0.041*"collect" '
  '+ 0.028*"collector" + 0.026*"claim" + 0.026*"money" + 0.025*"attempt" + '
  '0.023*"balance"'),
 (2,
  '0.230*"payment" + 0.130*"make" + 0.044*"month" + 0.032*"late" + '
  '0.027*"monthly" + 0.027*"plan" + 0.025*"pay" + 0.019*"job" + 0.014*"lose" + '
  '0.012*"additional"'),
 (3,
  '0.062*"apartment" + 0.057*"move" + 0.050*"lease" + 0.043*"rent" + '
  '0.034*"management" + 0.029*"vehicle" + 0.022*"deposit" + '
  '0.019*"apartment_complex" + 0.017*"month" + 0.014*"vacate"'),
 (4,
  '0.063*"many" + 0.030*"life" + 0.027*"clean" + 0.026*"everything" + '
  '0.025*"actually" + 0.021*"base" + 0.020*"refund" + 0.017*"government" + '
  '0.017*"thank" + 0.015*"automatically"'),
 (5,
  '0.066*"insurance" +

In [0]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -6.877939188956692

Coherence Score:  0.37974908309529004


In [0]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.264686  0.136677       1        1  21.595413
17    -0.229002 -0.233263       2        1  13.031426
16    -0.210323 -0.074477       3        1  10.306713
10    -0.192862  0.243967       4        1   9.521145
1     -0.194157 -0.052784       5        1   8.885978
11    -0.022351  0.105566       6        1   5.320463
12    -0.078619  0.134858       7        1   5.204011
18    -0.038674 -0.154664       8        1   3.721760
2     -0.037032 -0.172879       9        1   3.558878
19     0.051776 -0.011811      10        1   2.820120
5      0.078411  0.080335      11        1   2.706674
6      0.127119  0.107600      12        1   1.961586
4      0.135572 -0.005817      13        1   1.743915
7      0.075081  0.074606      14        1   1.707218
3      0.075997 -0.037557      15        1   1.703722
9      0.140313 -0.034868      16        1   1.413609
15     0.088595 -0.072599      17        1   1.402344
8      0.160216 -0.014458      18        1   1.195268
14     0.162719  0.004202      19        1   1.138103
0      0.171904 -0.022634      20        1   1.061655, topic_info=     Category         Freq             Term        Total  loglift  logprob
term                                                                      
2     Default  4058.000000             call  4058.000000  30.0000  30.0000
5     Default  4621.000000             debt  4621.000000  29.0000  29.0000
44    Default  1682.000000          payment  1682.000000  28.0000  28.0000
57    Default  3278.000000           credit  3278.000000  27.0000  27.0000
34    Default  2877.000000       collection  2877.000000  26.0000  26.0000
78    Default  2903.000000          account  2903.000000  25.0000  25.0000
43    Default  2761.000000              pay  2761.000000  24.0000  24.0000
30    Default  1320.000000             bill  1320.000000  23.0000  23.0000
17    Default  1564.000000              owe  1564.000000  22.0000  22.0000
41    Default  1256.000000             make  1256.000000  21.0000  21.0000
69    Default  2181.000000           report  2181.000000  20.0000  20.0000
73    Default  1125.000000          service  1125.000000  19.0000  19.0000
18    Default  1340.000000            phone  1340.000000  18.0000  18.0000
40    Default  1549.000000           letter  1549.000000  17.0000  17.0000
38    Default  1453.000000              get  1453.000000  16.0000  16.0000
147   Default  2141.000000          receive  2141.000000  15.0000  15.0000
0     Default   914.000000             bank   914.000000  14.0000  14.0000
72    Default  1318.000000              say  1318.000000  13.0000  13.0000
15    Default  1196.000000           number  1196.000000  12.0000  12.0000
23    Default  1670.000000             tell  1670.000000  11.0000  11.0000
410   Default   808.000000             loan   808.000000  10.0000  10.0000
77    Default  1786.000000            would  1786.000000   9.0000   9.0000
190   Default  1260.000000      information  1260.000000   8.0000   8.0000
87    Default   938.000000           charge   938.000000   7.0000   7.0000
48    Default  1708.000000             send  1708.000000   6.0000   6.0000
28    Default  1454.000000           agency  1454.000000   5.0000   5.0000
207   Default   952.000000            month   952.000000   4.0000   4.0000
489   Default   863.000000          request   863.000000   3.0000   3.0000
95    Default   849.000000             file   849.000000   2.0000   2.0000
517   Default   897.000000           amount   897.000000   1.0000   1.0000
...       ...          ...              ...          ...      ...      ...
1375  Topic20    53.892647        caller_id    54.835049   4.5280  -3.7015
783   Topic20    53.395172       individual    54.337574   4.5278  -3.7108
1576  Topic20    47.998104    family_member    48.940506   4.5259  -3.8174
1828  Topic20    47.812687           assign    48.755089   4.5258  -3